# Loading libraries and nodes layers

In [11]:
import pandas as pd
import numpy as np
from numba import jit
import sys, os




sys.path.append(r'..')
import MMSBM_library as sbm
from MMSBM_library.functions.utils import save_MMSBM_parameters,add_codes,load_EM_parameters

In [2]:
# Download MovieLens data.
# print("Downloading movielens data...")
# from urllib.request import urlretrieve
# 
# urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")


# import zipfile
# zip_ref = zipfile.ZipFile('movielens.zip', "r")
# zip_ref.extractall()
# print("Done. Dataset contains:")
# print(zip_ref.read('ml-100k/u.info'))

In [12]:
genres = ["unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western"]

In [13]:
df_users = pd.read_csv("ml-100k/u.user",sep="|",names=["uid","age","gender","occupation","zip_code"])
df_movies = pd.read_csv("ml-100k/u.item",sep="|",usecols=[0,1,2],names=["mid","title","date"], encoding='latin-1')


In [14]:
df_movies_genres =  pd.read_csv("ml-100k/u.item",sep="|",usecols=[0]+[i for i in range(5,24)],names =["mid"]+ genres, encoding='latin-1')

L = []

for i in df_movies_genres.iloc():
    L.append( "|".join([g for g in genres if i[g]!=0]))
df_movies["genres"] = L

In [15]:
df_movies

,mid,title,date,genres
0,1,Toy Story (1995),01-Jan-1995,Animation|Children's|Comedy
1,2,GoldenEye (1995),01-Jan-1995,Action|Adventure|Thriller
2,3,Four Rooms (1995),01-Jan-1995,Thriller
3,4,Get Shorty (1995),01-Jan-1995,Action|Comedy|Drama
4,5,Copycat (1995),01-Jan-1995,Crime|Drama|Thriller
...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,Drama
1678,1679,B. Monkey (1998),06-Feb-1998,Romance|Thriller
1679,1680,Sliding Doors (1998),01-Jan-1998,Drama|Romance
1680,1681,You So Crazy (1994),01-Jan-1994,Comedy


In [16]:
users = sbm.nodes_layer(nodes_info=df_users,K=10,nodes_name="uid")
#users.add_exclusive_metadata(10,"occupation")

movies = sbm.nodes_layer(nodes_info=df_movies,K=10,nodes_name="mid")
#movies.add_inclusive_metadata(123,"genres",10,separator="|")



In [17]:
m = movies["genres"]

ValueError: Metadata genres name not found

In [18]:
movies.df[str(m)+"_id"] = movies.df[str(m)]
for g in m.dict_codes:
    movies.df[str(m)+"_id"] = movies.df[str(m)+"_id"].str.replace(g,str(m.dict_codes[g]))

NameError: name 'm' is not defined

In [19]:
# links and neighbours
meta_name = "genres"
separator = "|"
df_dropna = movies.df.dropna(subset=[meta_name])
meta_list = movies.df[meta_name].values

observed = df_dropna[movies.node_type].values  # Nodes with known metadata
observed_id = df_dropna[movies.node_type + "_id"].values  # Nodes with known metadata


In [20]:
observed_id

array([   0,    1,    2, ..., 1679, 1680, 1681], dtype=int16)

In [11]:
try:
  os.mkdir("example_parameters")
except:
  pass
try:
  for f in range(1,6):
    os.mkdir(f"./example_parameters/kf_{f}")
except:
  pass

FileExistsError: [WinError 183] No se puede crear un archivo que ya existe: './kf_1'

# Using 5-fold cross validation with training and tests sets split in files 

In [ ]:
N_steps = 300
N_measures = 100

for f in range(1,6):

  df_links = pd.read_csv("ml-100k/u{}.base".format(f),sep="\t",names=["uid","mid","rating","time_stamp"])
  ratings = sbm.BiNet(df_links,"rating",nodes_a=users,nodes_b=movies)

  ratings.init_EM()

  for step in range(N_steps//N_measures):
    print(step*N_measures)
    ratings.EM_step(N_measures)
    if ratings.converges():
      print(f"Fold {f} converged! {step*N_measures} steps")
      break

#     if step% 10==0:
  print("saving!")
  #save_MMSBM_parameters(ratings,dir="./kf_{}".format(f),BiNet_json=True)


0


c:\Users\oscar\OneDrive\Documentos\MMSBM_library\..\MMSBM_library\functions\MMSBM_functions.py:78: RuntimeWarning: invalid value encountered in true_divide
  new_theta /= na.denominators


In [13]:
ratings.df

,uid,mid,rating,time_stamp,rating_id,uid_id,mid_id
0,1,1,5,874965758,4,0,0
1,1,2,3,876893171,2,0,1
2,1,4,3,876893119,2,0,3
3,1,5,3,889751712,2,0,4
4,1,6,5,887431973,4,0,5
...,...,...,...,...,...,...,...
79995,943,943,5,888639614,4,942,942
79996,943,1011,2,875502560,1,942,1010
79997,943,1067,2,875501756,1,942,1066
79998,943,1074,4,888640250,3,942,1073


In [14]:
help(sbm.BiNet.load_BiNet_from_json)

Help on method load_BiNet_from_json in module MMSBM_library:

load_BiNet_from_json(json_file, links, links_label, *, nodes_a=None, nodes_b=None, nodes_a_dir=None, nodes_b_dir=None, separator='\t') method of builtins.type instance
    Load a BiNet instance from a JSON file containing MMSBM parameters and link information.
    
    Parameters
    ----------
    json_file: str
        Path to the JSON files containing MMSBM parameters.
    
    links: str, pandas DataFrame
        DataFrame or directory containing the links between nodes_a and nodes_b and their labels.
    
    links_label: array-like
        Array-like object representing the labels corresponding to the links.
    
    nodes_a: nodes_layer, str, pd.DataFrame, None, default: None
        - If nodes_layer: Existing instance of the nodes_layer class representing the first layer.
        - If str or pd.DataFrame: If str, a name for the first layer. If pd.DataFrame, DataFrame with nodes and attributes.
        - If None: The 

In [15]:
df_links["rating"]

0        5
1        3
2        3
3        3
4        5
        ..
79995    5
79996    2
79997    2
79998    4
79999    3
Name: rating, Length: 80000, dtype: int64

In [ ]:
# Loading data from files

mean_accuracy = 0
for f in range(1,6):
  # Loading data from files and calculating accuracy for each fold
  # Mean accuracy is computed across all 5 folds
  df_links = pd.read_csv(f"ml-100k/u{f}.base",sep="\t",names=["uid","mid","rating","time_stamp"])
  df_test = pd.read_csv(f"ml-100k/u{f}.test",sep="\t",names=["uid","mid","rating","time_stamp"])

  ratings = sbm.BiNet.load_BiNet_from_json("./example_parameters/kf_{}/BiNet_data.json".format(f),links=df_links,links_label="rating",nodes_a=users,nodes_b=movies)
  ratings.init_EM_from_directory(dir="./example_parameters/kf_{}".format(f))

  accuracy = ratings.get_accuracy(links=df_test)
  mean_accuracy += accuracy
  print(f"Fold {f} accuracy: {accuracy}")

print(f"Mean accuracy: {mean_accuracy/5}")

Fold 1 accuracy: 0.4341
Fold 2 accuracy: 0.43955
Fold 3 accuracy: 0.43525
Fold 4 accuracy: 0.4301
Fold 5 accuracy: 0.4239
Mean accuracy: 0.43258


# Creating a k-fold

In [14]:
# Read full dataset
df_links = pd.read_csv("ml-100k/u.data", sep="\t", names=["uid","mid","rating","time_stamp"])

# Create initial BiNet with full dataset
ratings = sbm.BiNet(nodes_a=users, nodes_b=movies, links=df_links, links_label="rating")

# Create 5 random folds
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# For each fold
for fold, (train_idx, test_idx) in enumerate(kf.split(df_links), 1):
    print(f"Processing fold {fold}")
    
    # Split into train/test
    df_train = df_links.iloc[train_idx]
    df_test = df_links.iloc[test_idx]
    
    # Initialize EM with training data for this fold
    ratings.links = df_links  # Keep full dataset
    ratings.init_EM(training=df_train)  # Initialize EM with training set
    
    # Run EM algorithm
    ratings.EM_step(100)
    
    # Save parameters and train/test indices
    save_MMSBM_parameters(ratings, dir=f"./kf_{fold}", BiNet_json=True)
    
    # Save indices for reproducibility
    np.save(f"./kf_{fold}/train_idx.npy", train_idx)
    np.save(f"./kf_{fold}/test_idx.npy", test_idx)
    
    print(f"Fold {fold} complete")






Processing fold 1


c:\Users\oscar\OneDrive\Documentos\MMSBM_library\..\MMSBM_library\functions\MMSBM_functions.py:78: RuntimeWarning: invalid value encountered in true_divide
  new_theta /= na.denominators


Fold 1 complete
Processing fold 2


c:\Users\oscar\OneDrive\Documentos\MMSBM_library\..\MMSBM_library\functions\MMSBM_functions.py:78: RuntimeWarning: invalid value encountered in true_divide
  new_theta /= na.denominators


Fold 2 complete
Processing fold 3


...

Fold 3 complete
Processing fold 4


c:\Users\oscar\OneDrive\Documentos\MMSBM_library\..\MMSBM_library\functions\MMSBM_functions.py:78: RuntimeWarning: invalid value encountered in true_divide
  new_theta /= na.denominators


Fold 4 complete
Processing fold 5


c:\Users\oscar\OneDrive\Documentos\MMSBM_library\..\MMSBM_library\functions\MMSBM_functions.py:78: RuntimeWarning: invalid value encountered in true_divide
  new_theta /= na.denominators


Fold 5 complete


In [10]:
# Read full dataset
df_links = pd.read_csv("ml-100k/u.data", sep="\t", names=["uid","mid","rating","time_stamp"])


# Create initial BiNet with full dataset
ratings_loaded = sbm.BiNet(nodes_a=users, nodes_b=movies, links=df_links, links_label="rating")

# Initialize list to store accuracies
test_accuracies = []

# For each fold
for fold in range(1,6):
    # Load test indices
    test_idx = np.load(f"./kf_{fold}/test_idx.npy")
    train_idx = np.load(f"./kf_{fold}/train_idx.npy")
    
    # Get train/test data
    df_train = df_links.iloc[train_idx]
    df_test = df_links.iloc[test_idx]
    
    
    # Load trained parameters from directory
    ratings_loaded.init_EM_from_directory(df_train,f"./kf_{fold}")
    
    # Get predicted labels for test set
    test_predictions = ratings_loaded.get_predicted_labels(df_test)
    test_labels = df_test['rating'].replace(ratings_loaded.dict_codes).values
    
    # Calculate accuracy 
    test_acc = ratings_loaded.get_accuracy(links=df_test)
    test_accuracies.append(test_acc)
    
    print(f"\nFold {fold} test accuracy: {test_acc:.4f}")

# Print average accuracy
print("\nAverage test accuracy across folds:")
print(f"{np.mean(test_accuracies):.4f} ± {np.std(test_accuracies):.4f}")






Fold 1 test accuracy: 0.4381

Fold 2 test accuracy: 0.4385

Fold 3 test accuracy: 0.4330

Fold 4 test accuracy: 0.4362

Fold 5 test accuracy: 0.4344

Average test accuracy across folds:
0.4360 ± 0.0021


In [43]:
ratings.dict_codes

{5: 4, 3: 2, 4: 3, 1: 0, 2: 1}

In [44]:
df_test['rating_id'] = df_test['rating'].replace(ratings.dict_codes)

In [45]:
df_test

,uid,mid,rating,time_stamp,rating_id,uid_id,mid_id
0,1,6,5,887431973,4,0,5
1,1,10,3,875693118,2,0,9
2,1,12,5,878542960,4,0,11
3,1,14,5,874965706,4,0,13
4,1,17,3,875073198,2,0,16
...,...,...,...,...,...,...,...
19995,458,648,4,886395899,3,457,647
19996,458,1101,4,886397931,3,457,1100
19997,459,934,3,879563639,2,458,933
19998,460,10,3,882912371,2,459,9


In [34]:
help(ratings.init_EM_from_directory)

Help on method init_EM_from_directory in module MMSBM_library:

init_EM_from_directory(training=None, dir='.') method of MMSBM_library.BiNet instance
    Initialize the Expectation Maximization (EM) algorithm to obtain the most plausible membership parameters of the
    Mixed-Membership Stochastic Block Model (MMSBM) using parameters saved in files located in a specified directory.
    
        Parameters
        ----------
        dir: str, default: "."
            Directory where the files with the MMSBM parameters will be loaded.
    
        training: pd.DataFrame, list, ndarray, default: None
            - If pd.DataFrame: DataFrame containing the training links and labels.
            - If list or ndarray: List or array containing the positions of the links list from self.df attribute.
            - If None: Uses self.links_training and self.labels_training.



In [33]:
help(sbm.BiNet.get_accuracy)

Help on function get_accuracy in module MMSBM_library:

get_accuracy(self, predicted_labels=None, test_labels=None, Pij=None, links=None, estimator='max_probability')
    Computes the predicted labels of the model given the MMSBM parameters. They can be measured by different estimators:
        -max_probability: The predicted label will be the most plausible label
        -mean: The predicted label will be the mean
    
    Parameters
    ----------
    predicted_labels: array-like, default:None.
        Array-like with the predicted labels ids given by the MMSBM. If None, predictions will be generated using
    the specified links and estimator.
    
    test_labels: array-like, default:None.
        List or array with the observed labels. If None, labels from self.labels_array are taken given pos_test_labels
    
    links: ndarray of 1 or 2 dimensions, pandas DataFrame, default: None
        Array with links for which label probabilities are computed.
        -If a 2d-array, the fir

In [20]:
ratings_loaded = sbm.BiNet.load_BiNet_from_json("./kf_1/BiNet_data.json",df_links,"rating")

In [ ]:
for kf in range(1,5+1):
    ratings_loaded = sbm.BiNet.load_BiNet_from_json(f"./kf_{kf}/BiNet_data.json",df_links,"rating",nodes_a=users,nodes_b=movies)
    print(ratings_loaded.get_accuracy())

AttributeError: 'BiNet' object has no attribute 'pkl'

In [28]:
help(sbm.BiNet.load_BiNet_from_json)

Help on method load_BiNet_from_json in module MMSBM_library:

load_BiNet_from_json(json_file, links, links_label, *, nodes_a=None, nodes_b=None, nodes_a_dir=None, nodes_b_dir=None, separator='\t') method of builtins.type instance
    Load a BiNet instance from a JSON file containing MMSBM parameters and link information.
    
    Parameters
    ----------
    json_file: str
        Path to the JSON files containing MMSBM parameters.
    
    links: str, pandas DataFrame
        DataFrame or directory containing the links between nodes_a and nodes_b and their labels.
    
    links_label: array-like
        Array-like object representing the labels corresponding to the links.
    
    nodes_a: nodes_layer, str, pd.DataFrame, None, default: None
        - If nodes_layer: Existing instance of the nodes_layer class representing the first layer.
        - If str or pd.DataFrame: If str, a name for the first layer. If pd.DataFrame, DataFrame with nodes and attributes.
        - If None: The 

In [23]:
!ls kf_1

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [24]:
movies.theta

array([[0.12416474, 0.18121249, 0.050266  , ..., 0.12726425, 0.00379079,
        0.12464478],
       [0.14150685, 0.01404552, 0.1787501 , ..., 0.0612481 , 0.07762488,
        0.10978741],
       [0.09969689, 0.19304208, 0.11129702, ..., 0.02361406, 0.02776982,
        0.20955826],
       ...,
       [0.04743274, 0.16685162, 0.01737885, ..., 0.18871853, 0.08817513,
        0.01839571],
       [0.09591178, 0.14473575, 0.08091529, ..., 0.10665839, 0.23128031,
        0.05592058],
       [0.13094461, 0.04345709, 0.21443839, ..., 0.19912541, 0.09883361,
        0.06809797]])

In [25]:
sbm.functions.utils.save_nodes_layer_dict(users)

In [26]:
sbm.functions.utils.save_nodes_layer_dict(movies)

In [27]:
!ls -ltr

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [17]:
help(sbm.nodes_layer.__init__)

Help on function __init__ in module MMSBM_library:

__init__(self, K, nodes_name, nodes_info, *, separator='\t', dict_codes=None, **kwargs)
    Initialization of the nodes_layer class
    
    Parameters
    ----------
    K : int
        Number of memberships groups for the layer.
    nodes_name : str
        Name of the nodes column in the nodes_layer class
    nodes_info : str or pandas DataFrame
        If it is a string, it is the directory of the file with the nodes information.
        If it is a DataFrame, it is the DataFrame with the nodes information.
    separator : str, default        
        Separator if the columns of the file that contains the nodes information
    dict_codes : dict, default None
        Dictionary with the integer id of the nodes. The key is the nodes' name and the value its id.


In [25]:
import json
with open(".", "r") as f:
  print(f)
  data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './uid_data.json'

In [19]:
df_users = pd.read_csv("ml-100k/u.user",sep="|",names=["uid","age","gender","occupation","zip_code"])
# df_movies = pd.read_csv("ml-100k/u.item",sep="|",usecols=[0,1,2],names=["mid","title","date"], encoding='latin-1')


In [20]:
users2 = sbm.nodes_layer.load_nodes_layer_from_file(df_users,"./uid_data.json")

In [21]:
df_users

,uid,age,gender,occupation,zip_code,uid_id,occupation_id
0,1,24,M,technician,85711,0,19
1,2,53,F,other,94043,1,13
2,3,23,M,writer,32067,2,20
3,4,24,M,technician,43537,3,19
4,5,33,F,other,15213,4,13
...,...,...,...,...,...,...,...
938,939,26,F,student,33319,938,18
939,940,32,M,administrator,02215,939,0
940,941,20,M,student,97229,940,18
941,942,48,F,librarian,78209,941,10


In [22]:
users.df

,uid,age,gender,occupation,zip_code,uid_id,occupation_id
0,1,24,M,technician,85711,0,19
1,2,53,F,other,94043,1,13
2,3,23,M,writer,32067,2,20
3,4,24,M,technician,43537,3,19
4,5,33,F,other,15213,4,13
...,...,...,...,...,...,...,...
938,939,26,F,student,33319,938,18
939,940,32,M,administrator,02215,939,0
940,941,20,M,student,97229,940,18
941,942,48,F,librarian,78209,941,10


In [23]:
np.all(users2.df["occupation_id"] == users.df["occupation_id"])

True

In [24]:
data2.keys()

NameError: name 'data2' is not defined

In [25]:
for m in data["metadata_exclusives"]:
    print(m)
    users2.add_exclusive_metadata(**m)

{'meta_name': 'occupation', 'lambda_val': 10, 'N_atts': 21, 'dict_codes': {'technician': '19', 'other': '13', 'writer': '20', 'executive': '6', 'administrator': '0', 'student': '18', 'lawyer': '9', 'educator': '3', 'scientist': '17', 'entertainment': '5', 'programmer': '14', 'librarian': '10', 'homemaker': '8', 'artist': '1', 'engineer': '4', 'marketing': '11', 'none': '12', 'healthcare': '7', 'retired': '15', 'salesman': '16', 'doctor': '2'}}


In [26]:
df_movies2 =  df_movies#.drop("genres_id", axis='columns')


In [27]:
movies2 = sbm.nodes_layer.load_nodes_layer_from_file(df_movies2,"./mid_data.json")

In [28]:
import json
with open("./mid_data.json", "r") as f:
  data2 = json.load(f)

In [29]:
for m in data2["metadata_inclusives"]:
    movies2.add_inclusive_metadata(**m)

In [30]:
movies2.df

,mid,title,date,genres,mid_id,genres_id
0,1,Toy Story (1995),01-Jan-1995,Animation|Children's|Comedy,0,0|1|2
1,2,GoldenEye (1995),01-Jan-1995,Action|Adventure|Thriller,1,3|4|5
2,3,Four Rooms (1995),01-Jan-1995,Thriller,2,5
3,4,Get Shorty (1995),01-Jan-1995,Action|Comedy|Drama,3,3|2|6
4,5,Copycat (1995),01-Jan-1995,Crime|Drama|Thriller,4,7|6|5
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,Drama,1677,6
1678,1679,B. Monkey (1998),06-Feb-1998,Romance|Thriller,1678,10|5
1679,1680,Sliding Doors (1998),01-Jan-1998,Drama|Romance,1679,6|10
1680,1681,You So Crazy (1994),01-Jan-1994,Comedy,1680,2


In [31]:
movies.df

,mid,title,date,genres,mid_id,genres_id
0,1,Toy Story (1995),01-Jan-1995,Animation|Children's|Comedy,0,0|1|2
1,2,GoldenEye (1995),01-Jan-1995,Action|Adventure|Thriller,1,3|4|5
2,3,Four Rooms (1995),01-Jan-1995,Thriller,2,5
3,4,Get Shorty (1995),01-Jan-1995,Action|Comedy|Drama,3,3|2|6
4,5,Copycat (1995),01-Jan-1995,Crime|Drama|Thriller,4,7|6|5
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,Drama,1677,6
1678,1679,B. Monkey (1998),06-Feb-1998,Romance|Thriller,1678,10|5
1679,1680,Sliding Doors (1998),01-Jan-1998,Drama|Romance,1679,6|10
1680,1681,You So Crazy (1994),01-Jan-1994,Comedy,1680,2


In [32]:
meta = users2["occupation"]

In [33]:
meta.qka

AttributeError: 'exclusive_metadata' object has no attribute '_qka'

In [10]:
sbm.BiNet.load_BiNet_from_json("./kf_1/BiNet_data.json", df_links, "votes")

KeyError: 1325

In [11]:
import json
with open("./kf_1/BiNet_data.json", "r") as f:
  data = json.load(f)

In [ ]:
sbm.nodes_layer.create_simple_layer(Kb, self.df[nodes_b_name].unique(), nodes_b_name, dict_codes = dict_codes_b)

In [ ]:
users.df

In [ ]:
ratings.links_training.max(axis=0)

In [ ]:
def omega_comp_arrays(Na,Nb,p_kl,theta,eta,K,L,links_array,links_ratings):
    #new_omega = np.array(omega)
    omega = np.zeros((Na,Nb,K,L))
    for link  in range(len(links_ratings)):
        i = links_array[link][0]
        j = links_array[link][1]
        if i==944:print(i,j)
        if j==944:print(link,i,j)
        rating = links_ratings[link]
        omega[i,j,:,:] = p_kl[:,:,rating]*(np.expand_dims(theta[i,:], axis=1)@np.expand_dims(eta[j,:],axis=0))
        suma = omega[i,j,:,:].sum()
        omega[i,j,:,:] /= suma+1e-16
    return omega


In [ ]:
omega_comp_arrays(len(ratings.nodes_a),len(ratings.nodes_b),ratings.pkl,ratings.nodes_a.theta,ratings.nodes_b.theta,ratings.nodes_a.K,ratings.nodes_b.K,
                               ratings.links_training,ratings.labels_training)

1436 15 944


IndexError: index 944 is out of bounds for axis 0 with size 943

In [ ]:
ratings.pkl[:,:,0]*(np.expand_dims(users.theta[15,:], axis=1)@np.expand_dims(movies.theta[944,:],axis=0))

IndexError: index 944 is out of bounds for axis 0 with size 943

In [ ]:
movies.theta.shape

(943, 10)

In [ ]:
ratings

(10, 10, 5)
(943, 10)
(1682, 10)


In [25]:
help(sbm.BiNet.__init__)

Help on function __init__ in module MMSBM_library:

__init__(self, links, links_label, *, nodes_a=None, nodes_b=None, Ka=1, nodes_a_name='nodes_a', Kb=1, nodes_b_name='nodes_b', separator='\t', dict_codes=None, dict_codes_a=None, dict_codes_b=None)
    Initialization of a BiNet class
    
    Parameters
    -----------
    links: str, pandas DataFrame
       DataFrame or directory containing the links between nodes_a and nodes_b and their labels.
    
    links_label: str
        Name of the column where the labels are stored in the links DataFrame.
    
    nodes_a: nodes_layer, str, DataFrame, None, default: None
        One of the nodes layer that forms the bipartite network
        - If nodes_layer: Existing instance of the nodes_layer class representing the first layer.
        - If str or pd.DataFrame: If str, a directory containing the file information about nodes_a.
        - If pd.DataFrame, DataFrame with nodes_a information.
        - If None: A simple nodes_layer will be cr

In [30]:
from sklearn.model_selection import KFold
df_links = pd.read_csv("ml-100k/u.data".format(f),sep="\t",names=["uid","mid","rating","time_stamp"])
ratings = sbm.BiNet(df_links,"rating",nodes_a=users,nodes_b=movies)

kf = KFold(n_splits=5)


for i, (train_index, test_index) in enumerate(kf.split(df_links)):
  ratings.init_EM(training=train_index)

  for step in range(1):
    # print(step)
    ratings.EM_step()

#     if step% 10==0:
  print("saving!")
  save_MMSBM_parameters(ratings,dir="./kf_{}".format(i+1),BiNet_json=True)

saving!
saving!
saving!
saving!
saving!


In [ ]:
ratings.

In [16]:
for f in range(1,6):
  print(f)
  ratings = sbm.BiNet.load_BiNet_from_json("./kf_{}/BiNet_data.json".format(f),df_links,"rating",nodes_a=users,nodes_b=movies)
  print(len(ratings.nodes_a),len(ratings.nodes_b))
  ratings.init_EM_from_directory(df_links,"./kf_{}".format(f))

1
943 1682
./kf_1/pkl.npy
npy
2
943 1682
./kf_2/pkl.npy
npy
3
943 1682
./kf_3/pkl.npy
npy
4
943 1682
./kf_4/pkl.npy
npy
5
943 1682
./kf_5/pkl.npy
npy


In [17]:
def omega_comp_arrays(Na,Nb,p_kl,theta,eta,K,L,links_array,links_ratings):
    #new_omega = np.array(omega)
    omega = np.zeros((Na,Nb,K,L))
    for link  in range(len(links_ratings)):
        i = links_array[link][0]
        j = links_array[link][1]
        rating = links_ratings[link]
        omega[i,j,:,:] = p_kl[:,:,rating]*(np.expand_dims(theta[i,:], axis=1)@np.expand_dims(eta[j,:],axis=0))
        suma = omega[i,j,:,:].sum()
        omega[i,j,:,:] /= suma+1e-16
    return omega


In [20]:
len(users),len(movies)

(943, 1682)

In [21]:
omega_comp_arrays(len(users),len(movies),ratings.pkl,users.theta,movies.theta,users.K,movies.K,ratings.links_training,ratings.labels_training)


array([[[[4.86181565e-04, 8.61189093e-03, 6.04133174e-03, ...,
          7.48758938e-04, 5.83623805e-03, 2.56168579e-03],
         [1.94446423e-03, 1.23243147e-02, 2.20551606e-02, ...,
          6.72550464e-03, 3.60499141e-03, 1.26770959e-02],
         [9.46883478e-04, 1.11215663e-02, 1.24209057e-02, ...,
          1.89460501e-03, 1.66766575e-02, 1.52468703e-02],
         ...,
         [4.35129555e-04, 5.69508601e-03, 1.23069578e-02, ...,
          3.50674496e-03, 1.34917639e-02, 7.62206070e-03],
         [1.79040568e-03, 1.27615476e-02, 4.24313006e-02, ...,
          4.04463492e-03, 3.80888268e-03, 9.24905844e-03],
         [1.61793830e-04, 9.64022802e-04, 4.94665835e-03, ...,
          1.90465980e-04, 3.84003813e-05, 1.31988509e-03]],

        [[2.06586491e-03, 1.86123611e-05, 6.41311889e-04, ...,
          2.72248231e-03, 1.75050485e-03, 3.55942306e-04],
         [6.30293917e-03, 1.61156330e-04, 2.13276306e-03, ...,
          1.57553831e-02, 5.67352407e-02, 1.42929182e-02],
        

In [26]:
ratings.df.dtypes()

,uid,mid,rating,time_stamp,rating_id,uid_id,mid_id
0,1,1,5,874965758,4,0,0
1,1,2,3,876893171,2,0,1
2,1,3,4,878542960,3,0,2
3,1,4,3,876893119,2,0,3
4,1,5,3,889751712,2,0,4
...,...,...,...,...,...,...,...
79995,943,1067,2,875501756,1,942,1066
79996,943,1074,4,888640250,3,942,1073
79997,943,1188,3,888640250,2,942,1187
79998,943,1228,3,888640275,2,942,1227
